In [1]:
from pyspark.sql import SparkSession

In [2]:


spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("MyApp")   
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/20 18:08:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/20 18:08:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<h1> partition pruning<h1>

In [3]:
spark.conf.set("spark.sql.adaptive.enabled","false")

In [4]:
#checkeing AQE STATUS
spark.conf.get("spark.sql.adaptive.enabled")    

'false'

In [5]:
from pyspark.sql.functions import *

In [6]:
df = (
    spark.read
    .option("header", "true")         # if the CSV has headers
    .option("inferSchema", "true")    # enable schema inference
    .csv("/Users/bhushanchowdary/Documents/GitHub/pyspark/Whole_techniques/BigMart Sales.csv")
)


In [7]:
df.show(5,False)

+---------------+-----------+----------------+---------------+---------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type            |Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type      |Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+---------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|FDA15          |9.3        |Low Fat         |0.016047301    |Dairy                |249.8092|OUT049           |1999                     |Medium     |Tier 1              |Supermarket Type1|3735.138         |
|DRC01          |5.92       |Regular         |0.019278216    |Soft Drinks          |48.2692 |OUT018           |2009                     |Medium     |Tier 3              |Su

In [8]:
df.rdd.getNumPartitions()

1

In [9]:
spark.conf.set("spark.sql.files.maxPartitionBytes",131072)#partitiong the data based on the size of the data useually the recommended size for spark is 128 mb spark recommended (100-200) but maximum block sizes 
# between  128 so deafult size
#to make the partiions configurations based on the size one must change this or set the conf

In [10]:
df.rdd.getNumPartitions()

1

<h1> Repartitions <h1>

In [11]:
df = df.repartition(10)

In [12]:
df.rdd.getNumPartitions()

10

In [13]:
print(df.rdd.getNumPartitions())


10


In [14]:
from pyspark.sql import functions as F
# from pyspark.sql import functions as F

df_with_part = df.withColumn("partition_number", F.spark_partition_id())#this is too add the partition id for the row `F.spark_partition_id()`

# total count of non-null partition_number values
df_with_part.select(F.count("partition_number")).show()

# OR distribution of rows across partitions
df_with_part.groupBy("partition_number").count().show()


+-----------------------+
|count(partition_number)|
+-----------------------+
|                   8523|
+-----------------------+

+----------------+-----+
|partition_number|count|
+----------------+-----+
|               1|  854|
|               6|  851|
|               3|  852|
|               5|  850|
|               9|  853|
|               4|  853|
|               8|  852|
|               7|  851|
|               2|  854|
|               0|  853|
+----------------+-----+



<h1>Data writing for our partitions<h1>

In [15]:
df.write.format("parquet") \
    .mode("overwrite") \
    .save("/Users/bhushanchowdary/Documents/GitHub/pyspark/Whole_techniques/partitions/partition_01")


25/08/20 18:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/08/20 18:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/08/20 18:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/08/20 18:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/08/20 18:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [16]:
df_new = spark.read.parquet("/Users/bhushanchowdary/Documents/GitHub/pyspark/Whole_techniques/partitions/partition_01")

In [17]:
df_new.rdd.getNumPartitions()

10

In [18]:
df_new.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDB56|       8.75|         Regular|    0.074931201|Fruits and Vegeta...|187.6556|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|          938.778|
|          NCK17|       11.0|         Low Fat|    0.037971697|  Health and Hygiene|  38.948|           OUT045|                     2002|       NULL|              Tier 2|Superma

In [19]:
df.write.format("parquet").partitionBy("Outlet_Location_Type").mode("append").save("/Users/bhushanchowdary/Documents/GitHub/pyspark/Whole_techniques/partitions/partition_opt_01")

In [20]:
df_new = spark.read.parquet("/Users/bhushanchowdary/Documents/GitHub/pyspark/Whole_techniques/partitions/partition_opt_01")
df_new = df_new.filter(col("Outlet_Location_Type")=="Tier 1")

that is called static pruning we are applying pruning at the first step

## join optimizations 

Brodcast joins 10mb to 8gb 

In [21]:
spark.conf.set("spark.sql.adaptive.enabled","false")

In [ ]:
df_tran = spark.createDataFrame(
    (),
    (),
    (),
    (),
)